In [61]:
import pyscf
import pyscf.tools
import pyscf.cc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [41]:
def read_1rdm(fn, dimensions):
    df = pd.read_csv(fn, sep='\s+', names='coeff i j k l'.split())
    df = df.query('k==0 & l == 0')['coeff i j'.split()]
    
    dens = np.zeros(dimensions)
    dens[df.i.values-1, df.j.values -1] = df.coeff.values
    
    if not (dens_mo == dens_mo.conj().T).all():
        raise ValueError('Density matrix not hermitian.')
    return dens
dens_mo = read_1rdm('test-DM/CCDENSITIES', (18, 18))

In [51]:
def read_mocoeff(fn):
    mocoef = []
    
    # parse MOLDEN file
    with open(fn) as fh:
        lines = fh.readlines()
        
        is_MO = False
        seen_MO = 0
        for line in lines:
            line = line.strip()
            if line == '[MO]':
                is_MO = True
                continue
            if line.startswith('[]'):
                is_MO = False
                continue
            if not is_MO:
                continue
            
            if line.startswith('Ene='):
                seen_MO += 1
                continue
            if line.startswith('Occup='):
                continue
            if line.startswith('Spin='):
                spin = line.split()[-1]
                if spin != 'Alpha':
                    raise NotImplementedError('Only alpha spin supported')
                continue
            
            mocoef.append(float(line.split()[-1]))
    mocoef = np.array(mocoef).reshape(seen_MO, -1)

    # permute to MRCC order
    permutation = np.loadtxt('%s.perm' % fn).astype(int) - 1
        
    return mocoef[np.ix_(permutation, permutation)]
mo = read_mocoeff('test-DM/MOLDEN')

In [169]:
def test_case_pyscf():
    mol = pyscf.gto.M(atom='C 0 0 0; O 0 0 1.1', basis='6-31G')
    mf = pyscf.scf.RHF(mol).run()
    mycc = pyscf.cc.CCSD(mf).run()
    dm1 = mycc.make_rdm1()
    
    dm1_ao = np.einsum('pi,ij,qj->pq', mf.mo_coeff, dm1, mf.mo_coeff.conj())
    
    # Analytical evaluation
    print ('Total E_NE', np.matmul(mol.intor('int1e_nuc_sph'), dm1_ao).trace())
    for site in (0, 1):
        q = mol._atm.copy()
        q[site, 0] = 0
        a = pyscf.gto.getints('int1e_nuc_sph', q, mol._bas, mol._env)
        print ('Per-site E_NE', site, np.matmul(a, dm1_ao).trace())
    
    #Numerical evaluation
    grids = pyscf.dft.gen_grid.Grids(mol)
    grids.build()
    ao_value = pyscf.dft.numint.eval_ao(mol, grids.coords, deriv=0)
    rho = pyscf.dft.numint.eval_rho(mol, ao_value, dm1_ao, xctype='LDA')
    q = 0.
    for znuc, site in zip((6, 8), (0, 1)):
        thisq = -(grids.weights*rho*znuc/np.linalg.norm(grids.coords - mol.atom_coords()[site], axis=1)).sum()
        q += thisq
        print ('Per-site E_NE', site, thisq)
    print  ('Total E_NE', q)
test_case_pyscf()

converged SCF energy = -112.664778326693
E(CCSD) = -112.8696356050444  E_corr = -0.2048572783509469
Total E_NE -311.73470185329876
Per-site E_NE 0 -200.65909696970664
Per-site E_NE 1 -111.07560488359208
Per-site E_NE 0 -111.07560355660677
Per-site E_NE 1 -200.65909475029676
Total E_NE -311.7346983069035
